In [196]:
import json

In [197]:
with open('prediction_conll04_30/prediction.json.tmp', 'r') as f:
    data_tmp = json.load(f)

In [198]:
with open('prediction_conll04_30/prediction.json', 'r') as f:
    data = json.load(f)

In [199]:
len(data_tmp)

1303

In [200]:
len(data)

1303

In [201]:
with open('../data/core_scierc/scierc_30/fold_4/unlabeled_w_labels.json', 'r') as f:
    unlabeled = json.load(f)

In [202]:
len(unlabeled)

1303

In [203]:
count = 0
for row in unlabeled:
    if row['relations'] == []:
        count += 1

In [204]:
count

320

In [205]:
# def convert_doc_type_to_asp_type(atype, form):
#     if form == 'entity':
#         return atype.lower()
#     else:
#         split = atype.split('_')
#         if len(split) > 1:
#             return atype.split('_')[0].lower() + atype.split('_')[1]
#         else:
#             return atype.split('_')[0].lower()

In [206]:
def convert_doc_to_atoms(dataset, pred, verbose):
    atoms = []
    entities = pred['entities']
    relations = pred['relations']
    for ent in entities:
        etype = convert_doc_type_to_asp_type(dataset, ent['type'], 'entity')
        start = ent['start']
        end = ent['end']
        prob = 1
        if 'prob' in ent:
            prob = ent['prob']
        if verbose:
            c = f'atom(entity({etype},{start},{end}),"{prob}").'
        else:
            c = f'atom(entity({etype},{start},{end})).'
        atoms.append(c)
    for rel in relations:
        rtype = convert_doc_type_to_asp_type(dataset, rel['type'], 'relation')
        head = entities[rel['head']]
        head_start, head_end = head['start'], head['end']
        tail = entities[rel['tail']]
        tail_start, tail_end = tail['start'], tail['end']
        prob = 1
        if 'prob' in rel:
            prob = rel['prob']
        if verbose:
            c = f'atom(relation({rtype},{head_start},{head_end},{tail_start},{tail_end}),"{prob}").'
        else:
            c = f'atom(relation({rtype},{head_start},{head_end},{tail_start},{tail_end})).'
        atoms.append(c)
    return atoms

In [207]:
def convert_doc_type_to_asp_type(dataset, atype, form):
    dct = {
        'conll04': {
            'entity': {
                'Peop': 'peop',
                'Other': 'other',
                'Loc': 'loc',
                'Org': 'org'
            },
            'relation': {
                'Work_For': 'workFor',
                'OrgBased_In': 'orgbasedIn',
                'Live_In': 'liveIn',
                'Located_In': 'locatedIn',
                'Kill': 'kill'
            }
        },
        'ade': {
            'entity': {
                'Adverse-Effect': 'adverseEffect',
                'Drug': 'drug'
            },
            'relation': {
                'Adverse-Effect': 'adverseEffect'
            }
        },
        'scierc': {
            'entity': {
                'Task': 'task',
                'Method': 'method',
                'Material': 'material',
                'OtherScientificTerm': 'otherScientificTerm',
                'Metric': 'metric',
                'Generic': 'generic'
            },
            'relation': {
                'Used-for': 'usedFor',
                'Feature-of': 'featureOf',
                'Hyponym-of': 'hyponymOf',
                'Evaluate-for': 'evaluateFor',
                'Part-of': 'partOf',
                'Compare': 'compare',
                'Conjunction': 'conjunction'
            }
        }
    }
    return dct[dataset][form][atype]

In [147]:
t = {"tokens": ["In", "this", "paper", ",", "a", "novel", "method", "to", "learn", "the", "intrinsic", "object", "structure", "for", "robust", "visual", "tracking", "is", "proposed", "."], "entities": [{"type": "Generic", "start": 6, "end": 7, "prob": 0.9992194175720215}, {"type": "OtherScientificTerm", "start": 10, "end": 13, "prob": 0.9956142902374268}, {"type": "Task", "start": 14, "end": 17, "prob": 0.9977304339408875}], "relations": [{"type": "Used-for", "head": 0, "tail": 1, "prob": 0.9994097948074341}, {"type": "Used-for", "head": 0, "tail": 2, "prob": 0.9996190071105957}]}

In [148]:
convert_doc_to_atoms('scierc', t)

['atom(entity(generic,6,7),"0.9992194175720215").',
 'atom(entity(otherScientificTerm,10,13),"0.9956142902374268").',
 'atom(entity(task,14,17),"0.9977304339408875").',
 'atom(relation(usedFor,6,7,10,13),"0.9994097948074341").',
 'atom(relation(usedFor,6,7,14,17),"0.9996190071105957").']

In [173]:
def debug_case(i, dataset):
    atoms = convert_doc_to_atoms(dataset, unlabeled[i], True)
    pred_atoms = convert_doc_to_atoms(dataset, data[i], True)
    revised_atoms = convert_doc_to_atoms(dataset,data_tmp[i], True)
    
    print('grtr: ', atoms)
    print('pred: ', pred_atoms)
    print('revi: ', revised_atoms)

In [181]:
debug_case(21, 'scierc')

grtr:  ['atom(entity(generic,7,8),"1").', 'atom(entity(otherScientificTerm,11,13),"1").', 'atom(entity(material,14,16),"1").', 'atom(relation(usedFor,7,8,11,13),"1").', 'atom(relation(usedFor,14,16,7,8),"1").']
pred:  ['atom(entity(generic,4,5),"0.9986431002616882").', 'atom(entity(generic,6,8),"0.5896682739257812").', 'atom(entity(generic,7,8),"0.9957417249679565").', 'atom(entity(otherScientificTerm,11,13),"0.8259738683700562").', 'atom(entity(material,14,16),"0.9959904551506042").', 'atom(relation(conjunction,4,5,6,8),"0.9449558854103088").', 'atom(relation(usedFor,4,5,11,13),"0.5559950470924377").', 'atom(relation(usedFor,6,8,7,8),"0.9140499830245972").', 'atom(relation(usedFor,6,8,11,13),"0.9741834402084351").', 'atom(relation(usedFor,7,8,6,8),"0.9727578163146973").', 'atom(relation(usedFor,7,8,11,13),"0.9885531663894653").', 'atom(relation(usedFor,14,16,11,13),"0.9740813970565796").']
revi:  []


In [212]:
count = 0
for i in range(len(unlabeled)):
    atoms = convert_doc_to_atoms('scierc', unlabeled[i], False)
    
    pred_atoms = convert_doc_to_atoms('scierc', data[i], False)
    
    revised_atoms = convert_doc_to_atoms('scierc', data_tmp[i], False)
    
#     print('------------------------')
#     print(i)
#     print('grtr: ', atoms)
#     print('pred: ', pred_atoms)
#     print('revi: ', revised_atoms)
# #     print('grtr == pred: ', set(atoms) == set(pred_atoms))
#     print('revi != pred: ', set(revised_atoms) != set(pred_atoms))
    
#     if set(revised_atoms) != set(pred_atoms):
#         count += 1
        
    if set(atoms) != set(pred_atoms):
        print('------------------------')
        print(i)
        count += 1
        print('grtr: ', atoms)
        print('pred: ', pred_atoms)
        print('revi: ', revised_atoms)
#     print('grtr == revi: ', set(atoms) == set(revised_atoms))

------------------------
0
grtr:  ['atom(entity(material,0,1)).', 'atom(entity(otherScientificTerm,10,11)).', 'atom(entity(otherScientificTerm,17,21)).']
pred:  ['atom(entity(material,0,1)).', 'atom(entity(otherScientificTerm,10,11)).', 'atom(entity(otherScientificTerm,18,21)).', 'atom(relation(usedFor,10,11,0,1)).']
revi:  ['atom(entity(material,0,1)).', 'atom(entity(otherScientificTerm,10,11)).', 'atom(entity(otherScientificTerm,18,21)).', 'atom(relation(usedFor,10,11,0,1)).']
------------------------
1
grtr:  ['atom(entity(generic,1,2)).', 'atom(entity(otherScientificTerm,7,8)).', 'atom(entity(otherScientificTerm,9,11)).', 'atom(entity(otherScientificTerm,20,22)).', 'atom(entity(material,23,24)).', 'atom(entity(otherScientificTerm,26,28)).', 'atom(entity(material,29,30)).', 'atom(entity(material,32,33)).', 'atom(relation(conjunction,7,8,9,11)).', 'atom(relation(featureOf,26,28,29,30)).', 'atom(relation(hyponymOf,32,33,29,30)).']
pred:  ['atom(entity(otherScientificTerm,5,6)).', 'ato

pred:  ['atom(entity(task,10,12)).', 'atom(entity(otherScientificTerm,17,19)).', 'atom(entity(generic,22,23)).', 'atom(relation(usedFor,17,19,22,23)).']
revi:  ['atom(entity(task,10,12)).', 'atom(entity(otherScientificTerm,17,19)).', 'atom(entity(generic,22,23)).', 'atom(relation(usedFor,17,19,22,23)).']
------------------------
389
grtr:  ['atom(entity(generic,1,2)).', 'atom(entity(generic,11,12)).', 'atom(entity(material,14,16)).', 'atom(entity(generic,18,19)).', 'atom(relation(usedFor,1,2,11,12)).', 'atom(relation(usedFor,11,12,14,16)).']
pred:  ['atom(entity(generic,11,12)).', 'atom(entity(otherScientificTerm,14,16)).', 'atom(entity(generic,18,19)).', 'atom(relation(usedFor,11,12,14,16)).']
revi:  ['atom(entity(generic,11,12)).', 'atom(entity(otherScientificTerm,14,16)).', 'atom(entity(generic,18,19)).', 'atom(relation(usedFor,11,12,14,16)).']
------------------------
390
grtr:  ['atom(entity(generic,1,2)).', 'atom(entity(method,9,10)).', 'atom(entity(generic,12,13)).', 'atom(relat

pred:  ['atom(entity(otherScientificTerm,12,13)).', 'atom(entity(otherScientificTerm,21,23)).']
revi:  ['atom(entity(otherScientificTerm,12,13)).', 'atom(entity(otherScientificTerm,21,23)).']
------------------------
824
grtr:  ['atom(entity(method,5,8)).', 'atom(entity(material,15,17)).', 'atom(entity(method,27,29)).', 'atom(entity(material,31,34)).', 'atom(entity(generic,40,41)).', 'atom(relation(usedFor,15,17,5,8)).', 'atom(relation(usedFor,27,29,40,41)).', 'atom(relation(usedFor,31,34,27,29)).']
pred:  ['atom(entity(method,5,8)).', 'atom(entity(otherScientificTerm,15,17)).', 'atom(entity(otherScientificTerm,31,34)).', 'atom(entity(generic,40,41)).']
revi:  ['atom(entity(method,5,8)).', 'atom(entity(otherScientificTerm,15,17)).', 'atom(entity(otherScientificTerm,31,34)).', 'atom(entity(generic,40,41)).']
------------------------
825
grtr:  ['atom(entity(otherScientificTerm,13,15)).', 'atom(entity(otherScientificTerm,19,21)).']
pred:  ['atom(entity(material,13,15)).', 'atom(entity(ot

------------------------
1171
grtr:  ['atom(entity(task,3,5)).', 'atom(entity(otherScientificTerm,8,10)).', 'atom(relation(usedFor,8,10,3,5)).']
pred:  ['atom(entity(method,3,5)).', 'atom(entity(otherScientificTerm,8,10)).', 'atom(entity(otherScientificTerm,18,20)).', 'atom(relation(usedFor,3,5,8,10)).', 'atom(relation(usedFor,8,10,3,5)).']
revi:  ['atom(entity(method,3,5)).', 'atom(entity(otherScientificTerm,8,10)).', 'atom(entity(otherScientificTerm,18,20)).', 'atom(relation(usedFor,8,10,3,5)).']
------------------------
1173
grtr:  ['atom(entity(task,7,12)).', 'atom(entity(otherScientificTerm,10,12)).', 'atom(entity(otherScientificTerm,18,20)).', 'atom(entity(otherScientificTerm,24,26)).', 'atom(entity(otherScientificTerm,27,29)).', 'atom(relation(hyponymOf,24,26,18,20)).', 'atom(relation(conjunction,24,26,27,29)).', 'atom(relation(hyponymOf,27,29,18,20)).']
pred:  ['atom(entity(generic,1,2)).', 'atom(entity(otherScientificTerm,10,12)).', 'atom(entity(generic,15,16)).', 'atom(entity

In [192]:
count

144

In [193]:
len(unlabeled)

1303